In [20]:
import pandas as pd
import re
import numpy as np

## nomalice parameters
from unidecode import unidecode

import spacy  # For preprocessing
from scipy import spatial
from time import time  # To time our operations
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

from tqdm import tqdm

from deep_translator import GoogleTranslator
nlp = spacy.load("en_core_web_sm")

## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
## nomalice parameters
from unidecode import unidecode

In [21]:
df = pd.read_excel("final_DB.xlsx")

In [22]:
df.head()

,Bootcamps_unif,Link,job_title,education,job_history_clean,end_boot,end_boot_cat,etapa,educacion,max_univ,degree,educ_title
0,4Geeks,https://www.linkedin.com/in/yohangel,Analista Programador,"[{'education_org': 'curso: 4geeks academy', 'd...","[{'job_title': programming coordinator, 'start...",2016,end,1,"[{'education': 'curso: 4geeks academy', 'degre...",3,1,ingeniería informática
1,Acámica,https://www.linkedin.com/in/matías-rodríguez-b...,Student,"[{'summary': 'subjects approved 12/30 gpa:8', ...","[{'job_title': teacher assistant, 'start': 2016}]",2014,start,1,"[{'education': 'universidad de buenos aires', ...",8,1,"el arte de emprender desde américa latina, ent..."
2,Acámica,https://www.linkedin.com/in/ronny-ramirez-garcia,Vicepresidente de Operaciones y Tecnologia,[{'summary': 'el mba - máster en administració...,[{'job_title': head of payment methods and ele...,2016,end,1,[{'education': 'eneb - escuela de negocios eur...,6,3,"mba en administración y dirección de empresas,..."
3,Acámica,https://www.linkedin.com/in/florencia-toledano...,Socia,"[{'education_org': 'acamica', 'degrees': ['des...","[{'job_title': social, 'start': 2014}, {'job_t...",2015,end,1,"[{'education': 'acamica', 'degrees': 'desarrol...",8,1,"diseñadora gráfica, diseño gráfico"
4,Acámica,https://www.linkedin.com/in/rodolfo-alexander-...,Ingeniero,[{'education_org': 'universidad distrital fran...,"[{'job_title': software engineer developer, 's...",2016,end,1,[{'education': 'universidad distrital francisc...,6,1,"modernízate con angularjs (fundamentals), prog..."


In [23]:
for i in range(len(df))[0:387]:
    try:
        df["job_history_clean"][i] = df["job_history_clean"][i].replace("tle': ", "tle': '")
        df["job_history_clean"][i] = df["job_history_clean"][i].replace(", 'start", "', 'start")
        # df["job_history_clean"][i] = df["job_history_clean"][i].replace('"', "'")
    except:
        None


<ipython-input-23-96c426a0259e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["job_history_clean"][i] = df["job_history_clean"][i].replace("tle': ", "tle': '")
<ipython-input-23-96c426a0259e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["job_history_clean"][i] = df["job_history_clean"][i].replace(", 'start", "', 'start")


In [24]:
df.head()

,Bootcamps_unif,Link,job_title,education,job_history_clean,end_boot,end_boot_cat,etapa,educacion,max_univ,degree,educ_title
0,4Geeks,https://www.linkedin.com/in/yohangel,Analista Programador,"[{'education_org': 'curso: 4geeks academy', 'd...","[{'job_title': 'programming coordinator', 'sta...",2016,end,1,"[{'education': 'curso: 4geeks academy', 'degre...",3,1,ingeniería informática
1,Acámica,https://www.linkedin.com/in/matías-rodríguez-b...,Student,"[{'summary': 'subjects approved 12/30 gpa:8', ...","[{'job_title': 'teacher assistant', 'start': 2...",2014,start,1,"[{'education': 'universidad de buenos aires', ...",8,1,"el arte de emprender desde américa latina, ent..."
2,Acámica,https://www.linkedin.com/in/ronny-ramirez-garcia,Vicepresidente de Operaciones y Tecnologia,[{'summary': 'el mba - máster en administració...,[{'job_title': 'head of payment methods and el...,2016,end,1,[{'education': 'eneb - escuela de negocios eur...,6,3,"mba en administración y dirección de empresas,..."
3,Acámica,https://www.linkedin.com/in/florencia-toledano...,Socia,"[{'education_org': 'acamica', 'degrees': ['des...","[{'job_title': 'social', 'start': 2014}, {'job...",2015,end,1,"[{'education': 'acamica', 'degrees': 'desarrol...",8,1,"diseñadora gráfica, diseño gráfico"
4,Acámica,https://www.linkedin.com/in/rodolfo-alexander-...,Ingeniero,[{'education_org': 'universidad distrital fran...,"[{'job_title': 'software engineer developer', ...",2016,end,1,[{'education': 'universidad distrital francisc...,6,1,"modernízate con angularjs (fundamentals), prog..."


In [25]:
for i in range(len(df)):
    try:
        for a in range(1900,2030):
            df["job_history_clean"][i] = df["job_history_clean"][i].replace(f"{a}", f"'{a}'")
    except:
        None

<ipython-input-25-74a7fc53a3d4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["job_history_clean"][i] = df["job_history_clean"][i].replace(f"{a}", f"'{a}'")


In [26]:
df.job_history_clean[1]

"[{'job_title': 'teacher assistant', 'start': '2016'}]"

In [27]:
df.head()

,Bootcamps_unif,Link,job_title,education,job_history_clean,end_boot,end_boot_cat,etapa,educacion,max_univ,degree,educ_title
0,4Geeks,https://www.linkedin.com/in/yohangel,Analista Programador,"[{'education_org': 'curso: 4geeks academy', 'd...","[{'job_title': 'programming coordinator', 'sta...",2016,end,1,"[{'education': 'curso: 4geeks academy', 'degre...",3,1,ingeniería informática
1,Acámica,https://www.linkedin.com/in/matías-rodríguez-b...,Student,"[{'summary': 'subjects approved 12/30 gpa:8', ...","[{'job_title': 'teacher assistant', 'start': '...",2014,start,1,"[{'education': 'universidad de buenos aires', ...",8,1,"el arte de emprender desde américa latina, ent..."
2,Acámica,https://www.linkedin.com/in/ronny-ramirez-garcia,Vicepresidente de Operaciones y Tecnologia,[{'summary': 'el mba - máster en administració...,[{'job_title': 'head of payment methods and el...,2016,end,1,[{'education': 'eneb - escuela de negocios eur...,6,3,"mba en administración y dirección de empresas,..."
3,Acámica,https://www.linkedin.com/in/florencia-toledano...,Socia,"[{'education_org': 'acamica', 'degrees': ['des...","[{'job_title': 'social', 'start': '2014'}, {'j...",2015,end,1,"[{'education': 'acamica', 'degrees': 'desarrol...",8,1,"diseñadora gráfica, diseño gráfico"
4,Acámica,https://www.linkedin.com/in/rodolfo-alexander-...,Ingeniero,[{'education_org': 'universidad distrital fran...,"[{'job_title': 'software engineer developer', ...",2016,end,1,[{'education': 'universidad distrital francisc...,6,1,"modernízate con angularjs (fundamentals), prog..."


In [43]:
df.to_excel("final_DB_7.xlsx", index = False)